<!-- Simon-Style -->
<p style="font-size:19px; text-align:left; margin-top:    15px;"><i>German Association of Actuaries (DAV) — Working Group "Explainable Artificial Intelligence"</i></p>
<p style="font-size:25px; text-align:left; margin-bottom: 15px"><b>Use Case SOA GLTD Experience Study:<br>
Data preparation
</b></p>
<p style="font-size:19px; text-align:left; margin-bottom: 15px; margin-bottom: 15px">Guido Grützner (<a href="mailto:guido.gruetzner@quantakt.com">guido.gruetzner@quantakt.com</a>)


# Initialisation

This notebook will prepare a working version of the GLTD data file and provide some insight into its structure. We assume that the original data has been split and transformed to "feather" format using the prior notebook "data_initialisation.ipynb".

To run this notebook (or any of the other notebooks) adapt the variable `datadir` in line 2 of the code block below and insert the number of included parts of the whole study according to your compute resources. You can always restrict the size of the dataset later but loading the full dataset requires at least 32MB of main memory. If in doubt, stick with the default `parts=[0]` this will run with 8MB of main memory and give you about 6.4 million observations. If you are very patient or want to seriously test the power of your hardware set this to `parts[0:1]` or even to `parts[0:4]`. The last choice will load all the data of the study.  

In [20]:
import pandas as pd
pd.options.mode.copy_on_write = True

import os
import numpy as np

from IPython.display import display

In [21]:
# Adjust according to your setup
datadir = "d:/tmp/GLTD data/"
# don't change the name without good reasons
fn_out = "gltd09_13work.feather"

# number of parts in range(anzblock)
# if you want to make sense of the examples below you need to include 0
# if you don't care about them any subset of range(anzblock) is possible.
# e.g. parts[0:4] or parts[1:3] etc ...
parts = [0]

In [22]:
nm_in = ["gltd09_13_pt" + str(i) + ".feather" for i in parts]
for ifile in range(len(nm_in)):
    tt = [pd.read_feather(datadir + ifile) for ifile in nm_in]
    rawtbl = pd.concat(tt, axis=0)

nobs = rawtbl.shape[0]
display(f"Number of observations/records: {nobs}")
display(f"Number of inputs: {rawtbl.shape[1]}")
display(rawtbl.dtypes)

'Number of observations/records: 6594085'

'Number of inputs: 30'

Study_ID                                   int64
Elimination_Period                         int64
Calendar_Year                              int64
Calendar_Month                             int64
Duration_Month                             int64
Age_at_Disability                          int64
Diagnosis_Category                        object
OwnOccToAnyTransition                     object
Gender                                    object
Attained_Age                               int64
Mental_and_Nervous_Period                  int64
M_N_Limit_Transition                      object
Gross_Indexed_Benefit_Amount               int64
Industry                                  object
Indexed_Monthly_Salary                     int64
Taxability_of_Benefits                    object
Integration_with_STD                      object
Case_Size                                  int64
Residence_State                           object
COLA_Indicator                            object
Benefit_Max_Limit_Pr

Check for columns with NaNs. There shouldn't be any!

In [23]:
nan_cols = rawtbl.isna().sum() > 0
print(rawtbl.columns.values[nan_cols])
rawtbl = rawtbl.drop(columns=rawtbl.columns.values[nan_cols])

[]


Sanity check on binary labels.

In [24]:
# check whether event variables are either zero or one
# check whether any event occured
nm_label_cols = ["Actual_Recoveries", "Actual_Deaths", "Settlement_Counts", 
                 "Max_Out_Counts", "Limits_Count"]
for col in nm_label_cols:
    print(rawtbl[col].isin([0, 1]).all())
    print(rawtbl[col].sum())

True
92946
True
23203
True
5616
True
31862
True
8372


Check whether remaining termination events are mutually exclusive.

In [25]:
(rawtbl[["Actual_Deaths", "Actual_Recoveries", "Settlement_Counts", "Max_Out_Counts", 
         "Limits_Count"]].sum(axis=1) <= 1).all()   

True

Check for incorrect zero values in some numeric fields. 

In [26]:
nm_numcol = np.array(["Duration_Month", "Age_at_Disability", "Attained_Age", "Gross_Indexed_Benefit_Amount", 
                      "Indexed_Monthly_Salary", "Case_Size","Replacement_Ratio"])
tmp = rawtbl[nm_numcol].le(0).any(axis=0).to_numpy()
ltzero = rawtbl[nm_numcol[tmp]].le(0).sum(axis=0).to_numpy()
eqzero = rawtbl[nm_numcol[tmp]].eq(0).sum(axis=0).to_numpy()
pd.DataFrame({"variable": nm_numcol[tmp], "\u2264 0": ltzero, "=0": eqzero, "%": eqzero/rawtbl.shape[0]*100} )


,variable,≤ 0,=0,%
0,Indexed_Monthly_Salary,6053,6053,0.091794
1,Case_Size,315027,315027,4.777418
2,Replacement_Ratio,6053,6053,0.091794


# Data structure

For detailed definitions of the data fields, see 2019 Group Long-Term Disability Experience Study Preliminary Report, Appendix 3. 

Each unique Study_ID identifies an individual claim over time. The data contains one record for each month the claim is open during the study. For all claims which are already in-force at the study begin date, there is an additional record in the first month of the first study year, i.e. in Month 1 of 2009. This record will have a fractional exposure less than one while the other record for the same calendar month will be with full exposure, i.e. exposure equal to one. All subsequent records will show full exposure until termination of the claim, either due to an event or the termination of the study. At the end of the study period, which will be month 12 of calendar year 2017 and is not included in this dataset, any claims still open will again have two records, one with full exposure the other with fractional. If any claim is open for the full length of the study, its first and last fractional exposure will sum to one.

The following examples assume that part 0 is included in the file selection.

In [28]:
id_mt = rawtbl[ (rawtbl["Calendar_Month"]==1) & (rawtbl["Calendar_Year"]==2009)]["Study_ID"]
flg = (rawtbl["Study_ID"] == id_mt.iat[1])
tt = rawtbl[flg & (rawtbl["Calendar_Year"]==2009)][["Study_ID", "Calendar_Month", "Duration_Month", "Exposures"]]
pd.DataFrame(tt.sort_values(by="Duration_Month"))

,Study_ID,Calendar_Month,Duration_Month,Exposures
47,1698416,1,45,0.133333
343721,1698416,1,46,1.000000
366050,1698416,2,47,1.000000
415801,1698416,3,48,1.000000
230245,1698416,4,49,1.000000
201839,1698416,5,50,1.000000
76611,1698416,6,51,1.000000
407322,1698416,7,52,1.000000
201887,1698416,8,53,1.000000
233047,1698416,9,54,1.000000


If termination for death or recovery occurs, the respective record will have a full exposure.

In [29]:
id = rawtbl[(rawtbl["Calendar_Year"] == 2010) 
            & (rawtbl["Actual_Recoveries"] == 1)][ "Study_ID"]
tt = rawtbl[rawtbl["Study_ID"]==id.iat[1]]
tt = tt[["Study_ID", "Calendar_Month", "Duration_Month", 
         "Exposures", "Actual_Recoveries", "Actual_Deaths"]]
pd.DataFrame(tt.sort_values(by="Duration_Month"))

,Study_ID,Calendar_Month,Duration_Month,Exposures,Actual_Recoveries,Actual_Deaths
2587632,1883628,5,7,1.0,0,0
341197,1883628,6,8,1.0,0,0
115380,1883628,7,9,1.0,0,0
3785,1883628,8,10,1.0,0,0
334927,1883628,9,11,1.0,0,0
3513896,1883628,10,12,1.0,0,0
44170,1883628,11,13,1.0,0,0
152793,1883628,12,14,1.0,0,0
278,1883628,1,15,1.0,1,0


In [30]:
id = rawtbl[
    (rawtbl["Calendar_Year"] == 2010) & (rawtbl["Actual_Deaths"] == 1)]["Study_ID"]
tt = rawtbl[rawtbl["Study_ID"]==id.iat[1]]
tt = tt[["Study_ID", "Calendar_Month", "Duration_Month", "Exposures", 
         "Actual_Recoveries", "Actual_Deaths"]]
pd.DataFrame(tt.sort_values(by="Duration_Month"))

,Study_ID,Calendar_Month,Duration_Month,Exposures,Actual_Recoveries,Actual_Deaths
1684663,830655,3,7,1.0,0,0
72382,830655,4,8,1.0,0,0
480009,830655,5,9,1.0,0,0
3497370,830655,6,10,1.0,0,0
10332,830655,7,11,1.0,0,0
3404712,830655,8,12,1.0,0,0
403570,830655,9,13,1.0,0,0
10349,830655,10,14,1.0,0,1


Apparently, both claims entered the study in 2010 after a waiting period of 6 months.
If termination of a claim occurs for another reason (settlement, limit, max-out, exclusion) the observation of death or recovery is right-censored. In this case, a fractional exposure is shown. 

In [31]:
id = rawtbl[(rawtbl["Calendar_Year"] == 2010) & (rawtbl["Max_Out_Counts"] == 1)][ "Study_ID"]
tt = rawtbl[rawtbl["Study_ID"]==id.iat[6]]
tt = tt[["Study_ID", "Calendar_Month", "Duration_Month", "Exposures", "Max_Out_Counts"]]
pd.DataFrame(tt.sort_values(by="Duration_Month"))

,Study_ID,Calendar_Month,Duration_Month,Exposures,Max_Out_Counts
3663464,1979013,1,131,0.6,0
252233,1979013,1,132,1.0,0
148899,1979013,2,133,1.0,0
3452543,1979013,3,134,1.0,0
3327399,1979013,4,135,1.0,0
3400576,1979013,5,136,1.0,0
363179,1979013,6,137,1.0,0
3455843,1979013,7,138,1.0,0
255786,1979013,8,139,1.0,0
71461,1979013,9,140,1.0,0


The claim above was terminated due to "max-out" after 15 days (0.5 = 15/30) in the 146th month after commencement of benefits.

# Handling of fractional exposures

Fractional exposures contribute only little to the total exposure of this study.  

In [12]:
tbl = rawtbl[rawtbl["Exposures"] < 1]
display(f"Records with fractional exposures: {
    (tbl["Exposures"].sum() / rawtbl["Exposures"].sum()*100).round(1)}%")

'Records with fractional exposures: 0.9%'


The following result shows that the vast majority of fractional exposures is due to entry into the study. In addition, a conspicuous number of fractional exposures are in each February. This is due to max-out terminations, although it is not clear why contractual limits should relate to calendar time. Since we only use the 2009–2013 data and the study actually extends to 2017 no fractional years occur for the end of the study.

In [13]:
(pd.crosstab(tbl["Calendar_Month"], tbl["Calendar_Year"], values=tbl["Exposures"], 
             aggfunc="sum", normalize=True, margins=True) *100).round(1)

Calendar_Year,2009,2010,2011,2012,2013,All
Calendar_Month,,,,,,
1,82.9,0.2,0.2,0.3,0.3,84.0
2,0.4,0.5,0.5,0.6,0.6,2.6
3,0.2,0.2,0.2,0.3,0.3,1.2
4,0.2,0.2,0.2,0.3,0.3,1.2
5,0.2,0.2,0.3,0.3,0.2,1.2
6,0.2,0.2,0.3,0.3,0.3,1.3
7,0.2,0.3,0.3,0.3,0.3,1.4
8,0.2,0.3,0.3,0.3,0.3,1.4
9,0.3,0.2,0.3,0.3,0.3,1.5


In [14]:
flg = ((rawtbl["Calendar_Month"]==2) & 
       (rawtbl["Calendar_Year"]==2010) & 
       (rawtbl["Exposures"]<1))
print(rawtbl[flg][["Settlement_Counts", "Max_Out_Counts", "Limits_Count"]].sum())

Settlement_Counts      0
Max_Out_Counts       438
Limits_Count           0
dtype: int64


Since we would like to get rid of fractional exposures, as next step, all records of month 1 of year 2009 will be excluded. Since terminations due to deaths and recoveries will get an exposure of one, no matter whether they entered the study fully on month 1 or partially, it is not possible to filter all partial exposures. Just filtering out partial exposures which did not lead to termination would introduce bias. This removal decreases the total experience but, given the large overall amount of data available, this is not a problem. 

In [15]:
flg = (rawtbl["Calendar_Month"]==1) & (rawtbl["Calendar_Year"]==2009)
rawtbl = rawtbl[ ~flg]

# Some data hygiene


* Some of the entries in `Residence_State` are not spelled consistently (and the variable contains also Canadian provinces).
* It is more convenient to have a single numeric value for calendar time instead split ones for year and month
* Since the sole focus is on recovery, all other termination reasons can be dropped.
* Categorical variables get the appropriate dtype.

In [16]:
# some more data hygiene
rawtbl["Residence_State"] = rawtbl["Residence_State"].str.upper()
# Make calendar year and month to a single time index
rawtbl["Month_of_Study"] = 12 * ((rawtbl["Calendar_Year"] - 2009) 
                                 + rawtbl["Calendar_Month"])
rawtbl.drop(["Calendar_Year", "Calendar_Month"], axis=1, inplace=True)
# these are not required for fitting and can be dropped
rawtbl.drop(["Max_Out_Counts", "Settlement_Counts", "Actual_Deaths", 
             "Limits_Count", "Exposures"], axis=1, inplace=True)

# classification of inputs
col_cat = {'Elimination_Period', 'Diagnosis_Category', 
            'OwnOccToAnyTransition', 'Gender','Mental_and_Nervous_Period', 
           "M_N_Limit_Transition", 'Industry', 
            'Taxability_of_Benefits', 'Integration_with_STD','Residence_State', 
           'COLA_Indicator', 'Benefit_Max_Limit_Proxy',
            'Original_Social_Security_Award_Status',
           'Updated_Social_Security_Award_Status'}

rawtbl[list(col_cat)] = rawtbl[list(col_cat)].astype("category")
rawtbl.reset_index(drop=True, inplace=True)

# Write working version of the data 

To conclude, we write the remaining data in a convenient format. To prevent information creep, ten percent of the claims was already stored in a separate ultimate hold-out file. This means all the loaded data is available for model selection and calibration, i.e. train and test.

In [17]:
rawtbl.to_feather(path=datadir + fn_out)

The total size of the train/test file in MiB is

In [18]:
print(f"{round(os.path.getsize(datadir + fn_out) / 2**20,1)}MiB")

211.3MiB
